### July 17, 2024 run

In [2]:
import ollama
import pandas as pd 
import numpy as np
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, f1_score

In [3]:
corpus = pd.read_json('../data/old_data_clean.json')

In [5]:
irrelevant_path =  '../data/relevant_irrelevant_combined_gemma.txt'
with open(irrelevant_path, 'r') as file:
    irrelevant_def = file.read()

In [ ]:
meta_statement = """We categorize articles that are related to issues of income inequality, changes in income or wealth, general economic conditions."""

In [ ]:
max_char = 8000 * 4

output_list = []

prompt = "Is the article relevant? Answer relevant or irrelevant, and why in 1 sentence."

for num in range(len(corpus)):
    text = 'Title: ' + corpus.loc[num,'title'] + '\n' + corpus.loc[num,'text'][:max_char]
    print(num)
    print(text[:20])
    
    response = ollama.chat(
        model="gemma2:72b",
        messages=[
            {
                "role": "system",
                "content": "You are a news classifier."
            },
            {
                "role": "user",
                "content": meta_statement,
            },
            {
                "role": "user",
                "content": "Read this definition: " + irrelevant_def,
            },
            {
                "role": "user",
                "content": "Read this article: " + text,
            },
            {
                "role": "user",
                "content": prompt
            
            },
        ],
    options={
    "seed": 101,
    "temperature": 0,
    "num_ctx": 8020,
    "num_threads": 30,
    #"num_predict": 10,
      }
    )
    output_list.append((response["message"]["content"]))
output_list

In [ ]:
corpus['content'] = output_list

In [ ]:
corpus['gen_code'] = corpus['content']
corpus.loc[corpus['content'].str.lower().str.startswith('irrelevant'), 'gen_code'] = 0
corpus.loc[corpus['content'].str.lower().str.startswith('relevant'), 'gen_code'] = 1

In [ ]:
columns = ['title', 'year', 'month', 'journal', 'code', 'weight', 'code_label', 'id', 'code', 'content', 'gen_code']
corpus = corpus[columns]

corpus.to_csv('../data/inequality_dataset_gemma2-27b-GemmaDefinition.csv')